## HeroesOfPymoli Assignment
*Eric Nordstrom  
UT Data Analysis & Visualization Boot Camp*

### Notes:
* This notebook comes from a starter notebook provided for this project by the boot camp. Instructions and the first code block below were written by boot camp staff. Everything else was written by me.

In [98]:
# Dependencies and Setup
import pandas as pd, numpy as np

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

# display purhcase data sample
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [8]:
pd.DataFrame({'Total Players': [purchase_data['SN'].nunique()]}, index=[''])

,Total Players
,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [17]:
# show that each row has a different purchase ID
purchases = purchase_data['Purchase ID']
purchases.nunique() == purchases.count()

True

In [152]:
# create formatting function (overengineered for practice)

_number_format = '{:,}'
_money0_format = '${:,.0f}'
_money2_format = '${:,.2f}'
_pct_format = '{:.2f}%'

_keep_lower = {
    'of',
    'the',
    'a',
    'in',
    'to',
    'per'
}  # words to ignore when capitalizing to title format

to_replace = {
    'pct': '%',
    'avg': 'average',
    'num': '#',
    'tot': 'total'
}  # abbreviations to replace

def _replace_func(col_name, to_replace):
    '''looping version of "str.replace"'''
    
    for substring in to_replace:
        col_name = col_name.replace(substring, to_replace[substring])
    
    return col_name

def _format_values(new_df, col_names, format_string):
    for col_name in col_names:
        new_df[col_name] = new_df[col_name].apply(lambda x: format_string.format(x))

def _better_title(s):
    '''same as str.title() but keeps all uppercase letters and ignores certain words'''
    
    words = s.split()
    L = len(words)
    
    return ' '.join(
        word
        if word in _keep_lower
        and i not in {0, L}
        or word[1:].lower() != word[1:]
        else word[0].upper() + word[1:]
        for i, word in enumerate(words)
    )

def formatted(df, num=None, money2=[], money0=[], percent=[], capitalize=True, underscores=False, replacements={}):
    f'''\
Format the column names and values of the data frame.
Use "money0" for integer dollar amounts; use "money2" to include cents.
Assume all unspecified columns are "num" format unless "num" argument is specified or the column values cannot be interpreted appropriately.
By default, replacements include {to_replace}.\
'''
    
    new_df = df.copy()  # prevent mutating the input
    
    # set up replacements data
    to_replace.update(replacements)
    
    # format values
    _format_values(new_df, money2, _money2_format)
    _format_values(new_df, money0, _money0_format)
    _format_values(new_df, percent, _pct_format)
    
    if num is None:
        
        others = money0 + money2 + percent
        
        for col_name in new_df.columns:
            if col_name not in others:
                try:
                    _format_values(new_df, [col_name], _number_format)
                except ValueError:
                    pass
                except TypeError:
                    pass
        
    else:
        _format_values(new_df, num, _number_format)
    
    
    # format column names
    
    if underscores:
        us_func = lambda col_name: col_name
    else:
        us_func = lambda col_name: ' '.join( col_name.split('_') )
        
    if capitalize:
        cap_func = lambda col_name: _better_title(col_name)
    else:
        cap_func = lambda col_name: col_name
    
    rename_func = lambda col_name: cap_func(
        _replace_func( us_func(col_name), to_replace )
    )
        
    return new_df\
    .rename(columns={
        col_name: rename_func(col_name)
        for col_name in new_df.columns
    })\
    .rename_axis(
        rename_func(new_df.index.name)
        if isinstance(new_df.index.name, str)
        else None
    )
    

In [153]:
# test formatting function

num = ['Purchase ID', 'Age', 'Item ID']
money2 = ['Price']

formatted(purchase_data.head(), num, money2)

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Bin
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",$3.53,20-24
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,$1.56,40+
2,2,Ithergue48,24,Male,92,Final Critic,$4.88,20-24
3,3,Chamassasya86,24,Male,100,Blindscythe,$3.27,20-24
4,4,Iskosia90,23,Male,131,Fury,$1.44,20-24


In [154]:
prices = purchase_data['Price']
col_order = [
    'Item ID',
    'avg_price',
    'Purchase ID',
    'tot_revenue'
]  # original names
num = ['num of unique items', 'num of purchases']
money2 = ['avg_price']
money0 = ['tot_revenue']

formatted(
    
    pd.DataFrame(
        purchase_data[['Item ID', 'Purchase ID']]
        .nunique()
    ).T
    .assign(
        avg_price=prices.mean(),
        tot_revenue=prices.sum()
    )
    [col_order]
    .rename(columns={
        'Item ID': 'num of unique items',
        'Purchase ID': 'num of purchases'
    })
    .set_index([['']]),
    
    num,
    money2,
    money0
    
)

,# of Unique Items,Average Price,# of Purchases,Total Revenue
,183,$3.05,780,"$2,380"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [155]:
by_gender = purchase_data.groupby('Gender')
counts = by_gender['SN'].nunique()

formatted(
    
    pd.DataFrame({
        'tot count': counts,
        'pct of players': counts / counts.sum() * 100
    })
    [['tot count', 'pct of players']],
    
    num=['tot count'],
    percent=['pct of players']
    
)

,Total Count,% of Players
Gender,,
Female,81,14.06%
Male,484,84.03%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [156]:
purchase_amounts = by_gender['Price']
avg_tot_purchase_per_person = purchase_data.groupby(['Gender', 'SN'])['Price'].sum().groupby('Gender').mean()

formatted(
    
    by_gender[['Purchase ID']].count()
    .rename(columns={'Purchase ID': 'purchase_count'})
    .assign(
        avg_purchase_price=purchase_amounts.mean(),
        tot_purchase_value=purchase_amounts.sum(),
        avg_tot_purchase_per_person=avg_tot_purchase_per_person
    ),
    
    money2=['avg_purchase_price', 'avg_tot_purchase_per_person'],
    money0=['tot_purchase_value']
    
)

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Gender,,,,
Female,113,$3.20,$362,$4.47
Male,652,$3.02,"$1,968",$4.07
Other / Non-Disclosed,15,$3.35,$50,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [160]:
cuts = [0, 10, 15, 20, 25, 30, 35, 40, np.infty]
labels = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']
bins = pd.cut(purchase_data.Age, cuts, right=False, labels=labels)
purchase_data = purchase_data.assign(age_range=bins)  # make bins into column--required (I think?) to do double-grouping for `by_bin_and_person` in Purchasing Analysis (Age)
by_bin = purchase_data.groupby('age_range')

formatted(

    by_bin[['SN']].nunique()
    .rename(columns={'SN': 'num_of_players'})
    .assign(pct_of_players=lambda df: df.num_of_players / df.num_of_players.sum() * 100),
    
    percent=['pct_of_players']
    
)

,# of Players,% of Players
Age Range,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [161]:
by_bin_and_person = purchase_data.groupby(['age_range', 'SN'])
purchase_amounts = by_bin['Price']

formatted(
    
    by_bin[['Purchase ID']].count()
    .rename(columns={'Purchase ID': 'purchase_count'})
    .assign(
        avg_purchase_price=purchase_amounts.mean(),
        total_purchase_value=purchase_amounts.sum(),
        avg_total_purchase_per_person=by_bin_and_person['Price'].sum().groupby('age_range').mean()
    )
    .assign(
        pct_of_purchases=lambda df: df.purchase_count / df.purchase_count.sum() * 100,
        pct_of_purchase_value=lambda df: df.total_purchase_value / df.total_purchase_value.sum() * 100
    ),
    
    money2=[
        'avg_purchase_price',
        'total_purchase_value',
        'avg_total_purchase_per_person'
    ],
    
    percent=[
        'pct_of_purchases',
        'pct_of_purchase_value'
    ],
    
    replacements={'avg': 'avg'}  # too long otherwise
    
)

,Purchase Count,Avg Purchase Price,Totalal Purchase Value,Avg Totalal Purchase per Person,% of Purchases,% of Purchase Value
Age Range,,,,,,
<10,23,$3.35,$77.13,$4.54,2.95%,3.24%
10-14,28,$2.96,$82.78,$3.76,3.59%,3.48%
15-19,136,$3.04,$412.89,$3.86,17.44%,17.35%
20-24,365,$3.05,"$1,114.06",$4.32,46.79%,46.81%
25-29,101,$2.90,$293.00,$3.81,12.95%,12.31%
30-34,73,$2.93,$214.00,$4.12,9.36%,8.99%
35-39,41,$3.60,$147.67,$4.76,5.26%,6.21%
40+,13,$2.94,$38.24,$3.19,1.67%,1.61%


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
